In [20]:
import openai
import os
import json
import requests
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/Users/mac/Desktop/hateslop/hw9/3-Agent-1/.env")
load_dotenv(dotenv_path=env_path)

client = openai.OpenAI()
MODEL = "gpt-4o-mini"

OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
if not OPENWEATHER_API_KEY:
    raise ValueError(f"OPENWEATHER_API_KEY가 .env 파일에 설정되지 않았습니다.")


In [21]:
def get_today_weather(city: str) -> str:
    """
    지정된 도시의 현재 날씨 정보를 가져옵니다.
    
    Args:
        city: 도시 이름 (예: "Seoul", "Tokyo", "New York")
    
    Returns:
        날씨 정보를 담은 JSON 문자열
        예: {"weather": "Clear", "temp": 25.5, "humidity": 60, "description": "맑음"}
    """
    if not isinstance(city, str) or len(city.strip()) == 0:
        return json.dumps({"error": "유효하지 않은 도시 이름입니다."})
    
    from dotenv import load_dotenv
    from pathlib import Path
    env_path = Path("/Users/mac/Desktop/hateslop/hw9/3-Agent-1/.env")
    load_dotenv(dotenv_path=env_path)
    api_key = os.getenv("OPENWEATHER_API_KEY")
    
    if not api_key:
        return json.dumps({"error": "API 키를 로드할 수 없습니다. .env 파일을 확인해주세요."})
    
    # OpenWeatherMap API 호출
    url = f"http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key.strip(),  # 공백 제거
        "units": "metric",  # 섭씨 온도 사용
        "lang": "kr"  # 한국어 응답
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # 필요한 정보 추출
        weather_info = {
            "city": data.get("name", city),
            "weather": data.get("weather", [{}])[0].get("main", "Unknown"),
            "description": data.get("weather", [{}])[0].get("description", "알 수 없음"),
            "temp": data.get("main", {}).get("temp"),
            "feels_like": data.get("main", {}).get("feels_like"),
            "humidity": data.get("main", {}).get("humidity"),
            "wind_speed": data.get("wind", {}).get("speed")
        }
        
        return json.dumps(weather_info, ensure_ascii=False)
        
    except requests.exceptions.HTTPError as e:
        status_code = response.status_code if 'response' in locals() else None
        if status_code == 401:
            return json.dumps({"error": "API 키가 유효하지 않거나 만료되었습니다. OpenWeatherMap에서 API 키를 확인하고 .env 파일에 올바르게 설정해주세요."})
        elif status_code == 404:
            return json.dumps({"error": f"'{city}' 도시를 찾을 수 없습니다."})
        else:
            return json.dumps({"error": f"API 요청 중 에러 발생 (상태 코드: {status_code}): {str(e)}"})
    except Exception as e:
        return json.dumps({"error": f"날씨 정보 조회 중 오류 발생: {e}"})

# JSON 스키마 정의
weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_today_weather",
        "description": "지정된 도시의 현재 날씨 정보(날씨 상태, 온도, 습도 등)를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨를 조회할 도시 이름 (예: Seoul, Tokyo, New York)"
                }
            },
            "required": ["city"]
        }
    }
}

# 사용 가능한 함수 매핑
available_functions = {
    "get_today_weather": get_today_weather
}


Agent가 사용자 질문을 받아 Tool을 호출하고 최종 답변을 생성하는 전체 과정을 구현

In [22]:
# 사용자 질문
messages = [{"role": "user", "content": "오늘 서울 날씨 어때?"}]
tools = [weather_tool_schema]

print("1단계: LLM에 Tool 호출 요청")
response = client.chat.completions.create(
    model=MODEL, 
    messages=messages, 
    tools=tools, 
    tool_choice="auto"
)
response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls
if tool_calls:
    print("LLM이 Tool 사용을 결정")
    print("\n2단계: 결정된 Tool을 실제로 실행")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        print(f"- 실행 함수: {function_name}({', '.join([f'{k}={v}' for k, v in function_args.items()])})")
        
        function_response = function_to_call(**function_args)
        print(f"- 실행 결과: {function_response}")
        
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
        })
    
    print("\n3단계: Tool 실행 결과를 바탕으로 최종 답변 생성")
    final_response = client.chat.completions.create(model=MODEL, messages=messages)
    print("\n" + "="*50)
    print("Agent의 최종 답변")
    print("="*50)
    agent_answer = final_response.choices[0].message.content
    print(agent_answer)
else:
    print("LLM이 Tool 사용을 결정하지 않았습니다.")
    final_response = client.chat.completions.create(model=MODEL, messages=messages)
    print("\n" + "="*50)
    print("Agent의 최종 답변")
    print("="*50)
    print(final_response.choices[0].message.content)


1단계: LLM에 Tool 호출 요청
LLM이 Tool 사용을 결정

2단계: 결정된 Tool을 실제로 실행
- 실행 함수: get_today_weather(city=Seoul)
- 실행 결과: {"city": "Seoul", "weather": "Clear", "description": "맑음", "temp": 7.76, "feels_like": 5.76, "humidity": 34, "wind_speed": 3.09}

3단계: Tool 실행 결과를 바탕으로 최종 답변 생성

Agent의 최종 답변
오늘 서울의 날씨는 맑습니다. 현재 기온은 약 7.8도이며, 체감 온도는 5.8도입니다. 습도는 34%이고, 바람 속도는 3.1m/s입니다. 날씨가 좋으니 외출하기에 좋은 하루입니다!


추가 테스트 케이스

In [23]:
# 다른 도시 테스트 예시
test_questions = [
    "부산 날씨 알려줘",
    "What's the weather like in Tokyo?",
    "뉴욕 오늘 날씨는?"
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print('='*60)
    
    messages = [{"role": "user", "content": question}]
    
    # 1단계: Tool 호출 요청
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    response_message = response.choices[0].message
    messages.append(response_message)
    
    # 2단계: Tool 실행
    tool_calls = response_message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            })
        
        # 3단계: 최종 답변 생성
        final_response = client.chat.completions.create(model=MODEL, messages=messages)
        print(final_response.choices[0].message.content)
    else:
        print(response_message.content)



질문: 부산 날씨 알려줘
현재 부산의 날씨 정보를 제공할 수 없습니다. 최신 날씨 정보를 확인하려면 기상청 웹사이트나 날씨 앱을 확인해보시기 바랍니다.

질문: What's the weather like in Tokyo?
The weather in Tokyo today is partly cloudy with a temperature of about 14.1°C. It feels like 12.6°C, with a humidity level of 37%. There's a light wind blowing at a speed of 2.6 km/h.

질문: 뉴욕 오늘 날씨는?
오늘 뉴욕의 날씨는 비가 오고 있으며, 기온은 약 6.6도입니다. 체감 온도는 약 4.4도이고, 습도는 55%입니다. 바람 속도는 3.1m/s입니다. 우산을 챙기세요!
